<h1><center>Peer-Graded Assignment (180031412)

# Pet Adoption

https://www.hackerearth.com/problem/machine-learning/pet-adoption-9-5838c75b/description/

## Data-Set
https://www.kaggle.com/mannsingh/hackerearth-ml-challenge-pet-adoption

# Import Libaries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Reading the Data-Sets

In [2]:
train = pd.read_csv(r"C:/Users/nalla/ML/Data-Sets/Pet-Adoption/train.csv")
test= pd.read_csv(r"C:/Users/nalla/ML/Data-Sets/Pet-Adoption/test.csv")

In [3]:
train.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [4]:
test.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,42.73,0,7
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,6.71,0,1
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0.24,41.21,0,7
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,0.29,8.46,7,1
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0.71,30.92,0,7


In [5]:
train.shape

(18834, 11)

In [6]:
test.shape

(8072, 9)

# Feature Engineering

## Finding Missing Values

In [7]:
train.isnull().sum()

pet_id               0
issue_date           0
listing_date         0
condition         1477
color_type           0
length(m)            0
height(cm)           0
X1                   0
X2                   0
breed_category       0
pet_category         0
dtype: int64

In [8]:
test.isnull().sum()

pet_id            0
issue_date        0
listing_date      0
condition       619
color_type        0
length(m)         0
height(cm)        0
X1                0
X2                0
dtype: int64

In [9]:
train.dtypes

pet_id             object
issue_date         object
listing_date       object
condition         float64
color_type         object
length(m)         float64
height(cm)        float64
X1                  int64
X2                  int64
breed_category    float64
pet_category        int64
dtype: object

In [10]:
test.dtypes

pet_id           object
issue_date       object
listing_date     object
condition       float64
color_type       object
length(m)       float64
height(cm)      float64
X1                int64
X2                int64
dtype: object

In [11]:
train['breed_category'].value_counts()

0.0    9000
1.0    8357
2.0    1477
Name: breed_category, dtype: int64

<html><p style="font-size:18px">observe that the condition feature in the train data-set contains 1477 null values and in breed_category feature the 2.0 conintains the 1477 values let's check whether those are same or not

In [12]:
a=train['breed_category'][(np.isnan(train['condition']))]
a.value_counts()

2.0    1477
Name: breed_category, dtype: int64

<html><p style="font-size:18px">combine test and train data-set

In [14]:
#copy all test id to create submission file
test_id=test['pet_id']
#save the train left
ntrain=train.shape[0]

In [16]:
ntrain

18834

In [17]:
#save target variable i.e label
y1=train['breed_category']
y2=train['pet_category']

In [18]:
#combine test and train data
traintest = pd.concat((train, test)).reset_index(drop=True)
traintest.drop(['breed_category','pet_category'], axis=1, inplace=True)

In [19]:
traintest_copy=traintest.copy()
traintest['condition'].value_counts()

1.0    9747
0.0    8966
2.0    6097
Name: condition, dtype: int64

In [20]:
traintest['condition'].fillna(-1,inplace=True)

In [21]:
traintest['condition'].value_counts()

 1.0    9747
 0.0    8966
 2.0    6097
-1.0    2096
Name: condition, dtype: int64

<html><p style="font-size:18px">Since all the pets are not found in the same season, it is an important feature so we find the issue month and the listing date and the time difference between issue_date and the listing_date

In [22]:
#the pd.to_datetime is used to convert the object to datetime eature
traintest['issue_date']=pd.to_datetime(traintest['issue_date'])
traintest['listing_date']=pd.to_datetime(traintest['listing_date'])

In [23]:
x=[]
for d in traintest['issue_date']:
    y=d.month
    x.append(y)
traintest['issue_month']=x

In [24]:
x=[]
for d in traintest['listing_date']:
    y=d.month
    x.append(y)
traintest['listing_month']=x

In [25]:
traintest.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,issue_month,listing_month
0,ANSL_69903,2016-07-10,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,7,9
1,ANSL_66892,2013-11-21,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,11,12
2,ANSL_69750,2014-09-28,2016-10-19 08:24:00,-1.0,Brown,0.15,40.90,15,4,9,10
3,ANSL_71623,2016-12-31,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,12,1
4,ANSL_57969,2017-09-28,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,9,11


In [26]:
#time difference between issue and listing date(new feature)

In [28]:
x=[]
for d in traintest['listing_date']:
    y=d.year+(d.month/12.0)+(d.day/365.0)
    x.append(y)
traintest['modified_listing_date']=x

In [29]:
x=[]
for d in traintest['issue_date']:
    y=d.year+(d.month/12.0)+(d.day/365.0)
    x.append(y)
traintest['modified_issue_date']=x


In [30]:
traintest['took_time']=abs(traintest['modified_listing_date']-traintest['modified_issue_date'])

In [33]:
traintest.head(20)

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,issue_month,listing_month,modified_listing_date,modified_issue_date,took_time
0,ANSL_69903,2016-07-10,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,7,9,2016.807534,2016.610731,0.196804
1,ANSL_66892,2013-11-21,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,11,12,2019.073973,2013.974201,5.099772
2,ANSL_69750,2014-09-28,2016-10-19 08:24:00,-1.0,Brown,0.15,40.90,15,4,9,10,2016.885388,2014.826712,2.058676
3,ANSL_71623,2016-12-31,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,12,1,2019.151826,2017.084932,2.066895
4,ANSL_57969,2017-09-28,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,9,11,2017.968721,2017.826712,0.142009
5,ANSL_52200,2017-10-25,2019-03-10 13:18:00,0.0,Red,0.92,29.56,0,7,10,3,2019.277397,2017.901826,1.375571
6,ANSL_75444,2018-11-06,2019-04-09 16:00:00,2.0,Brown Tabby,0.14,40.24,0,1,11,4,2019.357991,2018.933105,0.424886
7,ANSL_52759,2018-06-05,2018-09-02 17:04:00,0.0,White,0.15,25.48,7,1,6,9,2018.755479,2018.513699,0.241781
8,ANSL_74632,2015-10-10,2016-08-04 16:29:00,2.0,Brown,0.05,38.22,13,9,10,8,2016.677626,2015.860731,0.816895
9,ANSL_56464,2014-02-18,2017-03-17 11:38:00,2.0,Brown,0.55,21.26,13,9,2,3,2017.296575,2014.215982,3.080594


In [35]:
#split into test and train
train = traintest[:ntrain]
test = traintest[ntrain:]

In [36]:
train=train.drop(['pet_id','issue_date','listing_date','modified_issue_date'],axis=1)
test=test.drop(['pet_id','issue_date','listing_date','modified_issue_date'],axis=1)

In [37]:
train.select_dtypes(exclude='number').columns

Index(['color_type'], dtype='object')

In [38]:
train=pd.get_dummies(train,drop_first=True)
test=pd.get_dummies(test,drop_first=True)

In [39]:
(train.shape),(test.shape)

((18834, 64), (8072, 62))

<html><p style="font-size:18px"> By Observing the shape of both the train and test set we can say that the train has 2 more features
so we need to remove those 2 features in the train data-set

In [40]:
a=set(train.columns)-set(test.columns)

In [41]:
a

{'color_type_Black Tiger', 'color_type_Brown Tiger'}

In [42]:
train=train.drop(a,axis=1)

In [43]:
train.shape,test.shape

((18834, 62), (8072, 62))

In [44]:
#again combining
traintest = pd.concat((train, test)).reset_index(drop=True)

# Scaling using StandardScaler()

In [45]:
from sklearn import preprocessing
#column names
names = traintest.columns
#creating object
scaler = preprocessing.StandardScaler()
# Fit the data-set
traintest_scale = scaler.fit_transform(traintest)
traintest = pd.DataFrame(traintest_scale, columns=names)

In [46]:
train= traintest[:ntrain]
test = traintest[ntrain:]

In [47]:
from sklearn.model_selection import train_test_split
X1_train,X1_test,y1_train,y1_test=train_test_split(train,y2,test_size=0.2,random_state=44,shuffle=True)

In [52]:
from xgboost import XGBClassifier
model1 = XGBClassifier()
model1.fit(X1_train, y1_train)
#predicting the train data-set
train_pred1=model1.predict(train)
#validating the train data-set
vld1=model1.predict(X1_test)
#predicting the test data-set
output1=model1.predict(test)

In [53]:
train2 = pd.DataFrame(train, columns=names)
test2 = pd.DataFrame(test, columns=names)

In [54]:
train2['output1']=train_pred1
test2['output1']=output1

In [55]:
X2_train,X2_test,y2_train,y2_test=train_test_split(train2,y1,test_size=0.2,random_state=44)

In [56]:
model2 = XGBClassifier()
model2.fit(X2_train, y2_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [58]:
output2=model2.predict(test)
vld2=model2.predict(X2_test)

# Checking Accuracy

In [59]:
from sklearn.metrics  import f1_score
s1=f1_score(y1_test,vld1,average='weighted')
s2=f1_score(y2_test,vld2,average='weighted')
accuracy=100*((s1+s2)/2)
accuracy

90.51687962901731

In [60]:
sub_new=pd.DataFrame({
    "pet_id":test_id,
    "breed_category":output2,
    "pet_category":output1
})
sub_new.to_csv("sub_new.csv",index=False)